# Clusters and Tracking Data
The purpose of this notebook is to generate data with corresponding clusters and tracks for both $\pi^0$ and $\pi^\pm$ data sets

These data sets can be quickly visualized and then saved into NP format or pickled

# Strategy:
### Use Single Track Events for Now
#### Event Filtering
1) Load branches from $\pi^\pm$ and $\pi^0$ data
2) Search through events in loop and find selection criteria. Save indices of event, track, and cluster
3) Use these indices to create np array of all the necessary data

## To Do:
1) For now we use loops to select events but we want to vectorize this (maybe? maybe abandon this..)
2) Create larger loop structure to take events with multiple tracks and match clusters to these
3) vectorize geoID cell search using np.tile() and np.argwhere()

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
import uproot as ur
import awkward as ak
import time as t
import copy
import scipy.constants as spc
print("Awkward version: "+str(ak.__version__))
print("Uproot version: "+str(ur.__version__))

Awkward version: 1.2.2
Uproot version: 4.0.7


#### Data and Branches

In [7]:
track_branches = ['trackEta_EMB1', 'trackPhi_EMB1', 'trackEta_EMB2', 'trackPhi_EMB2', 'trackEta_EMB3', 'trackPhi_EMB3',
                  'trackEta_TileBar0', 'trackPhi_TileBar0', 'trackEta_TileBar1', 'trackPhi_TileBar1',
                  'trackEta_TileBar2', 'trackPhi_TileBar2']

event_branches = ["cluster_nCells", "cluster_cell_ID", "cluster_cell_E", 'cluster_nCells', "nCluster", "eventNumber",
                  "nTrack", "nTruthPart", "truthPartPdgId", "cluster_Eta", "cluster_Phi", 'trackPt', 'trackP',
                  'trackMass', 'trackEta', 'trackPhi', 'truthPartE', 'cluster_ENG_CALIB_TOT']

geo_branches = ["cell_geo_ID", "cell_geo_eta", "cell_geo_phi", "cell_geo_rPerp", "cell_geo_sampling"]

##### Make a calorimetry layer dictionary

In [8]:
sampling_layer_track_list = [1,1,2,2,3,3,12,12,13,13,14,14]
track_layer_dict = dict(zip(track_branches, sampling_layer_track_list))

for _key in track_branches:
    print(_key+': '+str(track_layer_dict[_key]))

trackEta_EMB1: 1
trackPhi_EMB1: 1
trackEta_EMB2: 2
trackPhi_EMB2: 2
trackEta_EMB3: 3
trackPhi_EMB3: 3
trackEta_TileBar0: 12
trackPhi_TileBar0: 12
trackEta_TileBar1: 13
trackPhi_TileBar1: 13
trackEta_TileBar2: 14
trackPhi_TileBar2: 14


## All Functions

In [22]:
def DeltaR(_coord1, _coord2):
    ''' Straight forward function, expects Nx2 inputs for both'''
    _DeltaCoords = _coord1 - _coord2
    return np.sqrt(_DeltaCoords[:,0]**2 + _DeltaCoords[:,1]**2) 

def track_av(_arr):
    ''' Expects a (6,2) np array for the barrel layers in order of eta, phi '''
    _av_Eta = np.sum(_arr[:,0])/6
    _av_Phi = np.sum(_arr[:,1])/6
    return np.array([_av_Eta, _av_Phi])

def find_max_dim(_events, _event_dict):
    ''' This function is designed to return the sizes of a numpy array such that we are efficient
    with zero padding. Please feel free to write this faster, it can be done. Notes: we add six
    to the maximum cluster number such that we have room for track info.
    Inputs:
        _events: filtered list of events to choose from in an Nx3 format for event, track, cluster index 
        _event_tree: the event tree dictionary
    Returns:
        3-tuple consisting of (number of events, maximum cluster_size, 5), 5 because of how we have structured
        the X data format in energyFlow to be Eta, Phi, Energy, sampling layer, track flag,
        _cluster_ENG_CALIB_TOT, turthPartE '''
    _nEvents = len(_events)
    _max_clust = 0
    for i in range(_nEvents):
        _evt = _events[i,0]
        _clust_idx = _events[i,2]
        _num_in_clust = _event_dict['cluster_nCells'][_evt][_clust_idx]
        if _num_in_clust > _max_clust:
            _max_clust = _num_in_clust

    return (_nEvents, _max_clust+6, 7)

def find_index_1D(_values, _unsorted):
    ''' This function is a vectorized helper function to return the 1D locations of elements in a 
    larger unsorted list (i.e. cell geo locations). It does this by repeating/tiling arrays so they
    have equal shape and then simply comparing where the elements are equal and returning a slice
    where the indices match.
    Inputs: 
        _values: the values for which to search for the locations of in _unsorted
        _unsorted: the larger un-sorted list of elements to match
    Returns:
        _locs: locations of _values in _unsorted respectively. '''
    _shape = (len(_values), len(_unsorted))
    _values_2d = np.repeat(_values, repeats=len(_unsorted)).reshape(_shape)
    _unsorted_2d = np.tile(_unsorted, reps=len(_values)).reshape(_shape)
    
    _idx_vec = np.argwhere(np.asarray(_values_2d == _unsorted_2d))
    return _idx_vec[:,1]

def track_coords(_idx, _event_dict, _track_dict, _track_branches):
    ''' Returns a list of numpy arrays which contain the track information in the order of
    EMB1->TileBar2. '''
    
    _num_tracks = _event_dict['nTrack'][_idx]
    _track_arr = np.empty((_num_tracks,6,2))
    
    for i in range(_num_tracks):    
        _coords = np.empty((12,))

        j = 0
        for _key in _track_branches:
            _coords[j] = _track_dict[_key][_idx][i]
            j += 1

    _track_arr[i] = _coords.reshape(6,2)
    
    return _track_arr

def dict_from_event_tree(_event_tree, _branches):
    ''' The purpose for this separate function is to load np arrays where possible. '''
    _special_keys = ["nCluster", "eventNumber", "nTrack", "nTruthPart"]
    _dict = dict()
    for _key in _branches:
        if _key in _special_keys:
            _branch = _event_tree.arrays(filter_name=_key)[_key].to_numpy()
        else:
            _branch = _event_tree.arrays(filter_name=_key)[_key]
        _dict[_key] = _branch
    return _dict

def dict_from_tree_branches(_tree, _branches):
    ''' Helper function to put event data in branches to make things easier to pass to functions,
    pretty self explanatory. '''
    _dict = dict()
    for _key in _branches:
        _branch = _tree.arrays(filter_name=_key)[_key]
        _dict[_key] = _branch
    return _dict

def dict_from_tree_branches_np(_tree, _branches):
    ''' Helper function to put event data in branches to make things easier to pass to functions,
    pretty self explanatory. This always returns np arrays in the dict. '''
    _dict = dict()
    for _key in _branches:
        _branch = np.ndarray.flatten(_tree.arrays(filter_name=_key)[_key].to_numpy())
        _dict[_key] = _branch
    return _dict

def find_central_clusters(_event_dict):
    ''' Inputs:
        dictionary with events to pull cluster centers from
    Returns:
        an array of event indices with clusters with eta < .7
    '''
    # list of event indices with one or more clusters in EMB1-3 or TileBar0-2
    _central_events = []
    _ak_cluster_cell_ID = _event_dict['cluster_cell_ID']
    
    for _evt_idx in range(len(_ak_cluster_cell_ID)):
        _cluster_Eta = event_dict['cluster_Eta'][_evt_idx].to_numpy()
        _cluster_Eta_mask = np.abs(_cluster_Eta) < .7
        
        if np.any(_cluster_Eta_mask):
            _central_events.append(_evt_idx)

    return np.array(_central_events)

#### Load Events

In [4]:
# events = '/fast_scratch/atlas_images/v01-45/pipm/'
# event_502 = ur.open(events+'user.angerami.24559744.OutputStream._000502.root')
# my_event=event_502
# event_502.keys()

In [10]:
events = '/fast_scratch/atlas_images/v01-45/'
my_event = ur.open(events+'pipm_small.root')
my_event.keys()

['EventTree;1', 'CellGeo;1']

In [12]:
event_tree = my_event["EventTree"]
event_tree.show(name_width=42,
                interpretation_width=60)

name                                       | typename                 | interpretation                                              
-------------------------------------------+--------------------------+-------------------------------------------------------------
runNumber                                  | int32_t                  | AsDtype('>i4')
eventNumber                                | int64_t                  | AsDtype('>i8')
lumiBlock                                  | int32_t                  | AsDtype('>i4')
coreFlags                                  | uint32_t                 | AsDtype('>u4')
mcEventNumber                              | int32_t                  | AsDtype('>i4')
mcChannelNumber                            | int32_t                  | AsDtype('>i4')
mcEventWeight                              | float                    | AsDtype('>f4')
nTruthPart                                 | int32_t                  | AsDtype('>i4')
G4PreCalo_n_EM                        

In [13]:
event_dict = dict_from_event_tree(event_tree, event_branches)

nEvents = len(event_dict['eventNumber'])
print('Number of events: '+str(nEvents))

Number of events: 160000


##### Investigate truthParticleE

In [9]:
_truthPartE = event_dict['truthPartE']
print(_truthPartE[0:4])
for i in range(len(_truthPartE)):
    if i == 1:
        print(len(_truthPartE[i]))
    if len(_truthPartE[i]) != 1:
        print('multiple truth E found')

[[13.2], [459], [5.05], [54.1]]
1


#### Load Tracking Information

In [14]:
track_dict = dict_from_tree_branches(event_tree, track_branches)

### Load Geometry

In [11]:
CellGeo_tree = my_event["CellGeo"]
CellGeo_tree.show(interpretation_width=60)

name                 | typename                 | interpretation                                              
---------------------+--------------------------+-------------------------------------------------------------
cell_geo_ID          | std::vector<uint64_t>    | AsJagged(AsDtype('>u8'), header_bytes=10)
cell_geo_sampling    | std::vector<uint16_t>    | AsJagged(AsDtype('>u2'), header_bytes=10)
cell_geo_eta         | std::vector<float>       | AsJagged(AsDtype('>f4'), header_bytes=10)
cell_geo_phi         | std::vector<float>       | AsJagged(AsDtype('>f4'), header_bytes=10)
cell_geo_rPerp       | std::vector<float>       | AsJagged(AsDtype('>f4'), header_bytes=10)
cell_geo_deta        | std::vector<float>       | AsJagged(AsDtype('>f4'), header_bytes=10)
cell_geo_dphi        | std::vector<float>       | AsJagged(AsDtype('>f4'), header_bytes=10)
cell_geo_volume      | std::vector<float>       | AsJagged(AsDtype('>f4'), header_bytes=10)
cell_geo_sigma       | std::vector<float> 

In [15]:
geo_dict = dict_from_tree_branches_np(CellGeo_tree, geo_branches)

NameError: name 'CellGeo_tree' is not defined

## Filter events

### Criteria
 - Single track events
 - Track Pt > 500 MeV
 - Cluster within < .2 DeltaR
 - Cluster Energy > 5%

#### First select barrel events
Notes: Only use central cluster technique. Abandon searching sampling layers as searching for clusters is orders of magnitude faster. (3.56 hours versus .12 minutes for small root file)
### TO DO:
 - Make this a python script and save the output! Too slow otherwise and the connection tears
 - Fix this to make it faster!!! Right now it's god awful
   - Still slow as heck. Just ignore this part of the code and skip right to the track selection cuts

In [26]:
t0 = t.time()
central_events = find_central_clusters(event_dict)
t1 = t.time()

print('Time in hours: '+str(round(((t1-t0)/3600), 2)))
print('Time in minutes: '+str(round( ((t1-t0)/60), 3)))
print('Time in seconds: '+str(round((t1-t0), 2) ))

Time in hours: 0.0
Time in minutes: 0.133
Time in seconds: 7.97


### Track Filter
#### Single Track

In [27]:
# create ordered list of events to use for index slicing
nEvents = len(event_dict['eventNumber'])
all_events = np.arange(0,nEvents,1,dtype=np.int32)

# SINGLE TRACK CUT
single_track_mask = event_dict['nTrack'] == np.full(nEvents, 1)
filtered_event = all_events[single_track_mask]
print('Single Track Indices, shape '+str(np.shape(filtered_event)))
print(filtered_event[0:20])

Single Track Indices, shape (97242,)
[ 0  1  2  3  4  5  7  8  9 12 13 14 16 17 20 21 23 24 25 26]


#### Central Tracks

In [28]:
trackEta_EMB1 = ak.flatten(track_dict['trackEta_EMB1'][filtered_event]).to_numpy()
central_track_mask = np.abs(trackEta_EMB1) < .7
filtered_event = filtered_event[central_track_mask]
print('Central track indices, shape '+str(np.shape(filtered_event)))
print(filtered_event[0:20])

Central track indices, shape (30820,)
[  1   2   3   4   5   7  12  16  24  38  39  40  42  70  92  97 107 116
 124 137]


#### Tracks with Clusters

In [29]:
nCluster = event_dict['nCluster'][filtered_event]
filtered_event_mask = nCluster != 0
filtered_event = filtered_event[filtered_event_mask]
print('Tracks with clusters, shape '+str(np.shape(filtered_event)))
print(filtered_event[0:20])

Tracks with clusters, shape (26708,)
[  1   2   3   4   5   7  12  16  24  38  40  42  70  92  97 107 116 124
 137 155]


#### Momentum Cuts, 500 MeV
This is basically useless but here for completeness

In [30]:
trackP = ak.flatten(event_dict['trackP'][filtered_event]).to_numpy()
filtered_event_mask = trackP > .5
filtered_event = filtered_event[filtered_event_mask]
print('Tracks with momentum cuts, shape '+str(np.shape(filtered_event)))
print(filtered_event[0:40])

Tracks with momentum cuts, shape (26708,)
[  1   2   3   4   5   7  12  16  24  38  40  42  70  92  97 107 116 124
 137 155 160 177 179 185 190 197 204 210 211 212 220 223 237 246 257 270
 271 283 288 290]


### Failure for now, please ignore
### Find closest cluster
##### Attempt to do this with high level array slicing

In [97]:
# pull awkward array slices 
cluster_Etas = event_dict["cluster_Eta"][filtered_event]
cluster_Phis = event_dict["cluster_Phi"][filtered_event]

trackEta = event_dict["trackEta"][filtered_event]
trackPhi = event_dict["trackPhi"][filtered_event]

Now solve for DeltaR

In [102]:
print(cluster_Etas[0:4])
print(cluster_Phis[0:4])
mysum = cluster_Etas + cluster_Phis
# mydiff = cluster_Etas - trackEta
print(mysum[0:4])

[[0.0556, 0.831], [-0.157], [0.607, 0.578, 0.689, 0.585], [-0.641, -0.849]]
[[-0.136, 0.943], [-1.26], [-2.29, -2.41, -2.57, -2.17], [-1.49, -1.49]]
[[-0.0809, 1.77], [-1.42], [-1.68, -1.83, -1.88, -1.59], [-2.14, -2.34]]


## Cluster Track Matching
#### Apply $\Delta R$ cut <.1
 - loop through track filtered events and find nearest cluster, then apply cut

In [23]:
# Note we have to keep this a list becuase it may be shorter than filtered events if critera are not met
event_indices = []

for _evt in filtered_event:
    
    # pull cluster number, don't need zero as it's loaded as a np array
    _nClust = event_dict["nCluster"][_evt]
    
    # pull coordinates of tracks and clusters from event
    # we can get away with the zeroth index because we are working with single track events
    _trackEta = np.full( (_nClust,), event_dict["trackEta"][_evt][0])
    _trackPhi = np.full( (_nClust,), event_dict["trackPhi"][_evt][0])
    _trackCoords = np.stack((_trackEta, _trackPhi), axis=1)
    
    _cluster_Eta = event_dict["cluster_Eta"][_evt].to_numpy()
    _cluster_Phi = event_dict["cluster_Phi"][_evt].to_numpy()
    _clusterCoords = np.stack((_cluster_Eta, _cluster_Phi), axis=1)
    
    # Compute Delta
    _DeltaR = DeltaR(_trackCoords, _clusterCoords)
    
    if np.min(_DeltaR) < .1:
        _clust_idx = np.argmin(_DeltaR)
        event_indices.append((_evt,0,_clust_idx))    

event_indices = np.array(event_indices)
print(event_indices[0:10])

[[ 1  0  0]
 [ 2  0  0]
 [ 3  0  0]
 [ 4  0  0]
 [ 5  0  0]
 [ 7  0  0]
 [16  0  1]
 [40  0  0]
 [70  0  0]
 [92  0  0]]


In [25]:
np.save('e_i.npy', event_indices)

## Find np.array Dimensions

In [55]:
max_dims = find_max_dim(event_indices, event_dict)
print(max_dims)

(22339, 1017, 7)


## Input Data

In [62]:
X = np.zeros(max_dims)
t0 = t.time()

for i in range(len(event_indices)):
    # pull all relevant indices
    _evt = event_indices[i,0]
    _track_idx = event_indices[i,1]
    _cluster_idx = event_indices[i,2]
    
    ## CLUSTERS ##
    # cluster data
    _cluster_nCells = event_dict['cluster_nCells'][_evt][_cluster_idx]
    _cluster_cell_ID = event_dict['cluster_cell_ID'][_evt][_cluster_idx].to_numpy()
    _nInClust = len(_cluster_cell_ID)
    _cluster_cell_E = event_dict['cluster_cell_E'][_evt][_cluster_idx].to_numpy()
    _cluster_Eta = event_dict['cluster_Eta'][_evt][_cluster_idx]
    _cluster_Phi = event_dict['cluster_Phi'][_evt][_cluster_idx]
    
    # cell geometry data
    _cell_geo_ID = geo_dict['cell_geo_ID']
    
    # have to do this in loops for now, will change soon
    ## FIX THIS !!!
    _cell_idx = np.zeros(_nInClust, dtype=np.int32)
    for j in range(_nInClust):
        _cell_ID = _cluster_cell_ID[j]
        _cell_idx[j] = np.where(_cell_ID == _cell_geo_ID)[0][0]

    _cluster_cell_Eta = geo_dict['cell_geo_eta'][_cell_idx]
    _cluster_cell_Phi = geo_dict['cell_geo_phi'][_cell_idx]
    _cluster_cell_sampling = geo_dict['cell_geo_sampling'][_cell_idx]*.1
    
    # input all the data (vectorized :-) )
    # note here we leave the fourth entry zeros (zero for flag!!!)
    # Normalize to cluster centers!!!
    X[i,0:_nInClust,0] = _cluster_cell_Eta - _cluster_Eta
    X[i,0:_nInClust,1] = _cluster_cell_Phi - _cluster_Phi
    X[i,0:_nInClust,2] = np.log(_cluster_cell_E)
    X[i,0:_nInClust,3] = _cluster_cell_sampling

    ## Truth ##
    _cluster_ENG_CALIB_TOT = event_dict['cluster_ENG_CALIB_TOT'][_evt][_cluster_idx]
    X[i,:,5] = np.log(_cluster_ENG_CALIB_TOT)
    _truthPartE = event_dict['truthPartE'][_evt][0]
    X[i,:,6] = np.log(_truthPartE)
    
    ## TRACKS ##
    # Solve for track energy
    _trackP = event_dict['trackP'][_evt][_track_idx]
    _trackMass = event_dict['trackMass'][_evt][_track_idx]
    _trackE = np.sqrt(_trackP**2 + _trackMass**2)
    
    # track coordinates (normalized to cluster center!!)
    _track_arr = np.zeros((6,5))
    _track_arr[0,0] = track_dict['trackEta_EMB1'][_evt][0] - _cluster_Eta
    _track_arr[0,1] = track_dict['trackPhi_EMB1'][_evt][0] - _cluster_Phi
    _track_arr[1,0] = track_dict['trackEta_EMB2'][_evt][0] - _cluster_Eta
    _track_arr[1,1] = track_dict['trackPhi_EMB2'][_evt][0] - _cluster_Phi
    _track_arr[2,0] = track_dict['trackEta_EMB3'][_evt][0] - _cluster_Eta
    _track_arr[2,1] = track_dict['trackPhi_EMB3'][_evt][0] - _cluster_Phi
    _track_arr[3,0] = track_dict['trackEta_TileBar0'][_evt][0] - _cluster_Eta
    _track_arr[3,1] = track_dict['trackPhi_TileBar0'][_evt][0] - _cluster_Phi
    _track_arr[4,0] = track_dict['trackEta_TileBar1'][_evt][0] - _cluster_Eta
    _track_arr[4,1] = track_dict['trackPhi_TileBar1'][_evt][0] - _cluster_Phi
    _track_arr[5,0] = track_dict['trackEta_TileBar2'][_evt][0] - _cluster_Eta
    _track_arr[5,1] = track_dict['trackPhi_TileBar2'][_evt][0] - _cluster_Phi
    
    # add the track flag of one
    _track_arr[:,4] = np.ones(6)
    
    # add sampling layer
    _track_arr[:,3] = np.array([.1,.2,.3,1.2,1.3,1.4])
    
    # energy
    _track_arr[:,2] = np.full((6,), np.log(_trackE)/6)
    
    # search for NULL track flags
    _track_eta_null_mask = np.abs(_track_arr[:,0]) > .7
    _track_phi_null_mask = np.abs(_track_arr[:,1]) >= np.pi
    _track_flag_null = np.logical_or(_track_eta_null_mask, _track_phi_null_mask)
    # where the flag is set to null, set values of energy and calo layer to zero
    if np.any(_track_flag_null):
        _track_arr[_track_flag_null,2] = 0
        _track_arr[_track_flag_null,3] = 0
        # re-adjust energy somehow
        # Fix this later !!
    
    X[i,_cluster_nCells:_cluster_nCells+6,0:5] = _track_arr
t1 = t.time()

print('Finished constructing X array! :-)')
print('Time: '+str(round( (t1-t0)/60, 1)) +' min')

<ipython-input-62-1bc0107f1358>:43: RuntimeWarning: divide by zero encountered in log
  X[i,:,5] = np.log(_cluster_ENG_CALIB_TOT)


Finished constructing X array! :-)
Time: 33.9 min


In [64]:
np.save('X_7_tracks_small.npy', X)

# Testing Things: Ignore Below

In [209]:
sorted_arr = np.arange(0,8,1)
unsorted_arr = np.array([3,4,2,2,1,5,0,0])
print(sorted_arr)
print(unsorted_arr)
indices = np.ndarray.flatten(np.argwhere(np.asarray(sorted_arr == unsorted_arr)))
print(indices)

my_mask = np.array([True, False, False, True])
my_array = np.ones(4)
my_array[my_mask] = 0
print(my_array)

[0 1 2 3 4 5 6 7]
[3 4 2 2 1 5 0 0]
[2 5]
[0. 1. 1. 0.]


### Vectorize cell ID search

##### Check vectorized array sort using np.tile()

In [103]:
unsorted_array = np.arange(8)
np.random.shuffle(unsorted_array)
_tags = np.array([2,3])
print('Finding: '+str(_tags))
print('In: '+str(unsorted_array))

Finding: [2 3]
In: [2 4 5 0 3 6 1 7]


In [117]:
my_find = find_index_1D(_tags, unsorted_array)
print(my_find)

[0 4]


In [113]:
# Create array of ID elements
_tags_2d = np.repeat(_tags, repeats=len(unsorted_array)).reshape((len(_tags), len(unsorted_array)))
print(_tags_2d)

# Create stacked list of what to compare to
unsorted_2d = np.tile(unsorted_array, reps=len(_tags)).reshape((len(_tags), len(unsorted_array)))
print(unsorted_2d)

# now use comparison
idx_vect = np.argwhere(np.asarray( _tags_2d == unsorted_2d ))[:,1]
print(idx_vect)

[[2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3]]
[[2 4 5 0 3 6 1 7]
 [2 4 5 0 3 6 1 7]]
[0 4]


In [85]:

unsorted_tiled = np.tile(unsorted_array, len(_tags))
print('Tags:')
print(tags_tiled)
print('Unsorted array')
print(unsorted_array)

ValueError: cannot reshape array of size 2 into shape (2,2)

##### Using np.where()

In [75]:
_tag_mask = np.where(unsorted_array == _tags)
print(_tag_mask)

(array([], dtype=int64),)


<ipython-input-75-379848478688>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  _tag_mask = np.where(unsorted_array == _tags)


##### Using argwhere

In [78]:
indices = np.ndarray.flatten(np.argwhere(np.asarray(unsorted_array == _tags)))
print(indices)

[]


<ipython-input-78-866f63f4dfa9>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  indices = np.ndarray.flatten(np.argwhere(np.asarray(unsorted_array == _tags)))


##### Check this speed against dictionaries

In [71]:
cellGeoDict = {}
_tag_idx = 0
for _cell_tag in geo_dict['cell_geo_ID'][0:10]:
    if _tag_idx <= 5:
        print(_cell_tag)
    cellGeoDict[_cell_tag] = _tag_idx
    _tag_idx += 1

# 740294660 this should be index three
print('For element 740294660 this produces index: '+str(cellGeoDict[740294660]))

740294656
740294658
740294660
740294662
740294664
740294666
For element 740294660 this produces index: 2


Testing random stuff

In [46]:
W = np.zeros((5,2))
W[:,1] = 1
print(W)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
